In [ ]:
import troponin as trp
import numpy as np
import matplotlib
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import cv2

raw_data_location = '/media/matthew/Seagate Expansion Drive/Thinfil_micrographs_and_coords/'

In [ ]:
imstring = "image_1.mrc"
im_location = raw_data_location + 'micrographs_final/' + imstring

image = trp.get_mrc_image(im_location)

im_small = cv2.resize(image, None, fx=0.2, fy=0.2, interpolation=cv2.INTER_AREA)

thick_l, thick_li, pct, edges = trp.find_actin(img=im_small, low_th=5,
                                               high_th=30, min_ll=40,
                                               erosion_reps=5)
long_li, long_lis, long_lines = trp.find_actin_ll(img=im_small, low_th=5, sigma=5,
                                               high_th=30, min_ll=40,
                                               true_min_ll=100)

fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(16,24))
ax[0,0].imshow(im_small, cmap='gray')
ax[0,1].imshow(edges)
ax[1,0].imshow(thick_li)
ax[1,1].imshow(thick_l)
ax[2,0].imshow(long_lis)
ax[2,1].imshow(long_li)
plt.show()
print(pct)
print(len(long_lines))

In [ ]:
im_smaller = cv2.resize(image, None, fx=2/30, fy=2/30, interpolation=cv2.INTER_AREA)

thick_l0, thick_li0, pct0, edges0 = trp.find_actin(img=im_smaller, low_th=5, sigma=0,
                                               high_th=30, min_ll=5, thickness=1, max_lg=10,
                                               threshld=50, erosion_reps=2)
long_li0, long_lis0, long_lines0 = trp.find_actin_ll(img=im_smaller, low_th=5, sigma=0,
                                               high_th=30, min_ll=5, thickness=1, max_lg=10,
                                               threshld=50, true_min_ll=10)

fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(16,24))
ax[0,0].imshow(im_smaller, cmap='gray')
ax[0,1].imshow(edges0)
ax[1,0].imshow(thick_li0)
ax[1,1].imshow(thick_l0)
ax[2,0].imshow(long_lis0)
ax[2,1].imshow(long_li0)
plt.show()
print(pct0)
print(len(long_lines0))

In [ ]:
from PIL import Image
nt_count=0
for i in range(1,453):
    imstring = "image_" + str(i) + ".mrc"
    im_location = raw_data_location + 'micrographs_final/' + imstring
    image = trp.get_mrc_image(im_location)
    im_small = cv2.resize(image, None, fx=0.2, fy=0.2, interpolation=cv2.INTER_AREA)
    thick_l, thick_li, pct, edges = trp.find_actin(img=im_small, low_th=5,
                                               high_th=30, min_ll=40,
                                               erosion_reps=5)
    rng = np.random.default_rng(i)
    for j in range(4):
        rx = rng.integers(low=0, high=im_small.shape[1], size=1)
        ry = rng.integers(low=0, high=im_small.shape[0], size=1)
        while thick_l[ry,rx] == 0:
            rx = rng.integers(low=0, high=im_small.shape[1], size=1)
            ry = rng.integers(low=0, high=im_small.shape[0], size=1)
            
        rx = rx[0]*5
        ry = ry[0]*5
        #print(rx, ry)
        nt_im = trp.grab_troponin(image, rx, ry, 150)
        
        nt_imstring = "nt_image_" + str(nt_count) + ".png"
        nt_count += 1
        nt_im_location = raw_data_location + 'not_troponin_samples/' + nt_imstring
        
        nt_im = np.asarray(nt_im)
        ntim= Image.fromarray(nt_im)
        if ntim.mode != 'RGB':
            ntim = ntim.convert('RGB')
        ntim.save(nt_im_location)
        
    

In [ ]:
from skimage.util import invert
from scipy import ndimage
from PIL import Image

def get_best_angle(angles):
    if len(angles) == 0:
        return 0
    elif len(angles) == 1:
        return angles[0]
    elif len(angles) == 2:
        if abs(angles[0]-angles[1]) > 2/3*np.pi:
            angles[0] = -angles[0]
        return np.mean(angles)
    else:
        d1 = abs(angles[1] - angles[0])
        d2 = abs(angles[2] - angles[1])
        d3 = abs(angles[2] - angles[0])
        if d1 < d2 and d1 < d3:
            return (angles[1]+angles[0])/2
        elif d2 < d1 and d2 < d3:
            return (angles[2]+angles[1])/2
        else:
            return (angles[2]+angles[0])/2
        

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=100, figsize=(20,500))
row = 0
for i in range(100):
    trp_string = raw_data_location + "troponin_images/itroponin_im_" + str(i) + ".png"
    oimage = cv2.imread(trp_string)
    oimage = cv2.cvtColor(oimage, cv2.COLOR_BGR2GRAY)
    fxy = 20/300
    image = cv2.resize(oimage, None, fx=fxy, fy = fxy, interpolation=cv2.INTER_AREA)
    image = invert(image)
    kernel_size = 3
    sigma = 0
    image = cv2.GaussianBlur(image,(kernel_size, kernel_size),sigma)
    
    b = np.percentile(image,70)
    image = image*(image > b)
    
        
    low_threshold = 0
    high_threshold = 100
    edges = cv2.Canny(image, low_threshold, high_threshold)
    
    rho = 2  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 10  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 5  # minimum number of pixels making up a line
    max_line_gap = 2  # maximum gap in pixels between connectable line segments

    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    
    line_lengths = []
    
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                line_lengths.append(np.sqrt((y2-y1)**2 + (x2-x1)**2))
        idx = np.argsort(line_lengths)
        lines = lines[idx]
        lines = lines[-3:]
    
    angles = []
    
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                a = np.arctan2((y2-y1),(x2-x1))
                #print(y2-y1, x2-x1)
                
                if a == np.pi/2:
                    a = -np.pi/2
                #elif a < 0:
                #    a += np.pi/2
                
                angles.append(a)
    
    angle = get_best_angle(angles)

    #img_rotated = ndimage.rotate(image, 180*angle/3.1415926, reshape=False, mode='reflect')
    oimg_rotated = ndimage.rotate(oimage, 180*angle/3.1415926, reshape=False, mode='reflect')
    
    #print(angles)

    """
    r_troponin = np.asarray(oimg_rotated)
    rtim= Image.fromarray(r_troponin)
    if rtim.mode != 'RGB':
        rtim = rtim.convert('RGB')
    rtim.save(raw_data_location + 'troponin_images/r_itroponin_im_' + str(i) + '.png')
    
    
    """
    image_x = oimage.shape[1]
    image_y = oimage.shape[0]
    middle_x = int(image_x/2)
    middle_y = int(image_y/2)

    x2 = int(middle_x + image_x/2 * np.cos(angle))
    y2 = int(middle_y + image_y/2 * np.sin(angle))
    x1 = 2*middle_x - x2
    y1 = 2*middle_y - y2
    
    
    line_image = np.copy(image) * 0
    #cv2.line(line_image, (x1,y1), (x2,y2), 255, 1)
    cv2.line(oimage, (x1,y1), (x2,y2), 255, 4)
    #cv2.line(line_image, (int(x1*fxy),int(y1*fxy)), (int(x2*fxy),int(y2*fxy)), 255, 1)


    
    

    
    # draw lines onto blank image
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                #cv2.circle(line_image,(x1,y1),2,(255,0,0),1)
                #cv2.circle(line_image,(x2,y2),2,(0,255,0),1)
                cv2.line(line_image,(x1,y1),(x2,y2),255,1)
            
    #ax[int(np.floor(i/5)),i%5].imshow(edges, cmap='gray')
    ax[row,0].imshow(oimage)
    ax[row,1].imshow(image)
    ax[row,2].imshow(edges)
    ax[row,3].imshow(line_image)
    #ax[2].imshow(line_image)
    row+=1
    
    

plt.show()

In [ ]:
trp_data = []
for i in range(1743):
    trp_string = raw_data_location + "troponin_images/r_itroponin_im_" + str(i) + ".png"
    image = cv2.imread(trp_string)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #image = np.asarray(image)
    if image.shape != (300,300):
        ix = image.shape[1]
        iy = image.shape[0]
        image = cv2.copyMakeBorder(image, int(np.floor((300-iy)/2)), int(np.ceil((300-iy)/2)), int(np.floor((300-ix)/2)), int(np.ceil((300-ix)/2)), cv2.BORDER_REFLECT)
    image = cv2.resize(image, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_AREA)
    image = np.asarray(image)
    image = image.flatten()
    trp_data.append(image)

trp_data = np.asarray(trp_data)
print(trp_data.shape)
    

In [ ]:
def sort_evals_descending(evals, evectors):
    """
    Sorts eigenvalues and eigenvectors in decreasing order. This function
    also aligns the first two eigenvectors to be in first two quadrants if
    the data is 2D (remember that any eigenvector's direction can be inverted
    and it is still an eigenvector with the same eigenvalue). 
    """

    index = np.flip(np.argsort(evals))
    evals = evals[index]
    evectors = evectors[:, index]
    if evals.shape[0] == 2:
        if np.arccos(np.matmul(evectors[:, 0], 1 / np.sqrt(2) * np.array([1, 1]))) > np.pi / 2:
            evectors[:, 0] = -evectors[:, 0]
        if np.arccos(np.matmul(evectors[:, 1], 1 / np.sqrt(2) * np.array([-1, 1]))) > np.pi / 2:
            evectors[:, 1] = -evectors[:, 1]
    return evals, evectors
    
def pca(X):
    """
    Performs PCA on multivariate data.

    Args:
    X (numpy array of floats) : Data matrix each column corresponds to a
                                different random variable

    Returns:
    (numpy array of floats)   : Data projected onto the new basis
    (numpy array of floats)   : eigenvectors
    (numpy array of floats)   : corresponding eigenvalues

    """

    # Subtract the mean of X
    X_bar = X - np.mean(X, axis=0, dtype=np.float32)
    # Calculate the sample covariance matrix
    cov_matrix = (1 / X.shape[0]) * np.matmul(X_bar.T, X_bar)
    # Calculate the eigenvalues and eigenvectors
    evals, evectors = np.linalg.eigh(cov_matrix)
    # Sort the eigenvalues in descending order
    evals, evectors = sort_evals_descending(evals, evectors)
    # Project the data onto the new eigenvector basis
    score = np.matmul(X, evectors)

    return score, evectors, evals, cov_matrix


trp_score, trp_evecs, trp_evals, cm = pca(trp_data)
trp_evecs = trp_evecs.T

In [ ]:
trp_pca_vars = []
trp_pca_vars_cum = [0]
trp_pca_cvsum = 0
for i in range(5625):
    trp_pca_vars.append(np.matmul(np.matmul(trp_evecs[i], cm),trp_evecs[i].T))
    trp_pca_cvsum += np.matmul(np.matmul(trp_evecs[i], cm),trp_evecs[i].T)
    if i > 0:
        trp_pca_vars_cum.append(trp_pca_vars_cum[i-1]+trp_pca_vars[i])
    else:
        trp_pca_vars_cum[i] = trp_pca_vars[i]

nth_var_sum = 0
for i in range(50):
    nth_var_sum += np.matmul(np.matmul(trp_evecs[i], cm),trp_evecs[i].T)/trp_pca_cvsum
    
print(nth_var_sum)
print(np.matmul(np.matmul(trp_evecs[0], cm),trp_evecs[0].T)/trp_pca_cvsum)
print(np.matmul(np.matmul(trp_evecs[1], cm),trp_evecs[1].T)/trp_pca_cvsum)

trp_pca_vars = np.array(trp_pca_vars)/trp_pca_cvsum*100
trp_pca_vars_cum = np.array(trp_pca_vars_cum)/trp_pca_cvsum*100

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(np.arange(5625), trp_pca_vars, '.', markersize=3)
ax.set_xlabel('Component')
ax.set_ylabel('Variance (\%)')
#fig.savefig('report/mnist_pca_vars.pdf', format='pdf', bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(np.arange(5625), trp_pca_vars_cum, linewidth=3)
#fig.savefig('report/mnist_pca_vars_cum.pdf', format='pdf', bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(5,5,figsize=(16,16))
for i in range(5):
    for j in range(5):
        ax[i,j].imshow(np.reshape(trp_evecs[i*5+j], (75,75)), cmap='gray')
        ax[i,j].axis('off')
#fig.savefig('report/mnist_pca1.pdf', format='pdf', bbox_inches='tight')
plt.show()
"""
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.imshow(np.reshape(trp_evecs[1], (75,75)))
ax.axis('off')
#fig.savefig('report/mnist_pca2.pdf', format='pdf', bbox_inches='tight')
plt.show()
"""
for i in range(25):
    trp_ev = np.reshape(trp_evecs[i], (75,75))
    trp_ev = np.interp(trp_ev, (trp_ev.min(), trp_ev.max()), (0,255))
    te_im= Image.fromarray(trp_ev)
    if te_im.mode != 'RGB':
        te_im = te_im.convert('RGB')
    te_im.save(raw_data_location + 'troponin_images/troponin_ev_' + str(i) + '.png')

In [ ]:
X_mean = np.mean(trp_data, axis=0)
fig, ax = plt.subplots(figsize=(8,8))
ax.imshow(X_mean.reshape(75,75), cmap='gray')
plt.show()

In [ ]:
e1 = invert(np.reshape(trp_evecs[2], (75,75)))
e1 = np.interp(e1, (e1.min(), e1.max()), (0,255))
e1 = e1.astype(np.uint8)
print(e1.min(), e1.max())
imstring = "image_3.mrc"
im_location = raw_data_location + 'micrographs_final/' + imstring
image = trp.get_mrc_image(im_location)
im_small = cv2.resize(image, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_AREA)

In [ ]:
MIN_MATCH_COUNT = 10

sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(e1,None)
kp2, des2 = sift.detectAndCompute(im_small,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = e1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    im_small = cv2.polylines(im_small,[np.int32(dst)],True,255,3, cv2.LINE_AA)

else:
    print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(e1,kp1,im_small,kp2,good,None,**draw_params)

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(img3, 'gray')
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=50, figsize=(20,250))

for i in range(100):
    trp_string = raw_data_location + "troponin_images/r_itroponin_im_" + str(i) + ".png"
    oimage = cv2.imread(trp_string)
    oimage = cv2.cvtColor(oimage, cv2.COLOR_BGR2GRAY)
    oimage = [j[:] for j in oimage[50:250]]
    oimage = np.asarray(oimage)
    fxy = 50/300
    image = cv2.resize(oimage, None, fx=fxy, fy = fxy, interpolation=cv2.INTER_AREA)
    image = invert(image)
    kernel_size = 3
    sigma = 0
    image = cv2.GaussianBlur(image,(kernel_size, kernel_size),sigma)
    
    b = np.percentile(image,70)
    image = image*(image > b)

    ax[int(np.floor(i/2)),2*(i%2)].imshow(oimage)
    ax[int(np.floor(i/2)),2*(i%2)+1].imshow(image)
    
    
plt.show()